# DATA SETUP AS AN API POINT (GITHUB API)
### Reducing and pre-processing data as the data is only used to create a lake house 

Data (kaggle URL) - https://www.kaggle.com/datasets/ahmedaliraja/e-commerece-sales-data-2023-24

    cURL:
        curl -L -o ~/Downloads/e-commerece-sales-data-2023-24.zip\
        https://www.kaggle.com/api/v1/datasets/download/ahmedaliraja/e-commerece-sales-data-2023-24

In [1]:
import pandas as pd
import os
import shutil

In [2]:
df=pd.read_csv("sales.csv")

# Data Exploration

In [3]:
print("Number of rows:",len(df))

Number of rows: 6758125


In [4]:
df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3


In [5]:
df.describe()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice
count,6.758125e+06,6.758125e+06,6.758125e+06,6.758125e+06,6.758125e+06,6.758125e+06,6758125.0
mean,3.379063e+06,1.199972e+01,4.939567e+04,2.265474e+02,1.300401e+01,2.996787e-02,0.0
std,1.950903e+06,6.632689e+00,2.850504e+04,1.304744e+02,7.209701e+00,6.398096e-02,0.0
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.0
25%,1.689532e+06,6.000000e+00,2.470700e+04,1.140000e+02,7.000000e+00,0.000000e+00,0.0
50%,3.379063e+06,1.200000e+01,4.941400e+04,2.270000e+02,1.300000e+01,0.000000e+00,0.0
75%,5.068594e+06,1.800000e+01,7.407500e+04,3.400000e+02,1.900000e+01,0.000000e+00,0.0
max,6.758125e+06,2.300000e+01,9.875900e+04,4.520000e+02,2.500000e+01,2.000000e-01,0.0


We see the the TotalPrice is Null, we will trnasform it in Fabric (ETL process)

In [6]:
# Drop null as we have huge amount of data and the peoject is focues on ELT and warehouse architecture
df=df.dropna()

In [7]:
# Sample data
df=df.sample(n=1200)

In [8]:
# Sorting data for incremental data loading (Optional)

df=df.sort_values(by='SalesDate')

In [9]:
# Data Directory
os.mkdir("data")
os.mkdir("data/incrementaldata")

In [10]:
# Save Sales data
df[:1000].to_csv("data/sales.csv")
df[1000:].to_csv("data/incrementaldata/sales.csv")

# Move other data files to data directory
for file in os.listdir():
    try:
        f=file.split(".")
        if f[1] =="csv" and f[0]!="sales":
            shutil.copy(file,f"data/{file}")
    except:
        pass